## Entitlement regression for all suffixes

In [ ]:
import os

import geopandas
import intake
import ipywidgets
import matplotlib.pyplot as plt
import numpy
import pandas
import sklearn.preprocessing
import sklearn.linear_model
import sklearn.pipeline
import sklearn.utils

import laplan
import utils

cat = intake.open_catalog("../catalogs/*.yml")

In [ ]:
# Census tracts
tracts = cat.census_tracts.read().set_index("GEOID")

In [ ]:
prefix_list = laplan.pcts.VALID_PCTS_PREFIX
suffix_list = laplan.pcts.VALID_PCTS_SUFFIX

remove_prefix = ["ENV"]
remove_suffix = [
    "EIR",
    "IPRO",
    "CA",
    "CATEX",
    "CPIO",
    "CPU",
    "CRA"
    "FH",
    "G",
    "HD",
    "HPOZ",
    "ICO",
    "K",
    "LCP",
    "NSO",
    "RFA",
    "S",
    "SN",
    "SP",
    "ZAI",
]

prefix_list = [x for x in prefix_list if x not in remove_prefix]
suffix_list = [x for x in suffix_list if x not in remove_suffix]

In [ ]:
# Merge the census data with the entitlements counts:
joined = utils.entitlements_per_tract(
    prefix_list=prefix_list,
    suffix_list=suffix_list,
    verbose=True,
    big_case_threshold=20,
    return_big_cases=False,
)

In [ ]:
joined.head()

In [ ]:
joined = geopandas.GeoDataFrame(
    joined, 
    geometry=tracts.geometry,
)

In [ ]:
include_entitlements = (
    joined[suffix_list]
    .sum()
    .sort_values(ascending=False)
    .head(30)
    .index
)
include_entitlements

In [ ]:
column = "TOC"
geopandas.GeoDataFrame(
    joined.astype({column: "float"}).assign(geometry=tracts.geometry),
    geometry="geometry",
).plot(figsize=(16,16), column=column)

In [ ]:
cols=[
    "medhhincome",
    "pct_pop_renter",
    "pct_zero_veh_workers",
    "density",
    "pct_whitenonhisp"
]

In [ ]:
model = sklearn.pipeline.Pipeline([
    ("scaler", sklearn.preprocessing.StandardScaler(with_mean=True)),
    ("regressor", sklearn.linear_model.PoissonRegressor(fit_intercept=True, tol=1.e-6))
])

In [ ]:
rs = numpy.random.RandomState(1)

def fit_model(variables, target, norm=1, samples=100): 
    def get_coefs(model):
        return model["regressor"].coef_

    samples = numpy.array([
        get_coefs(model.fit(*sklearn.utils.resample(
            variables,
            (target/norm).replace([numpy.nan, numpy.inf], 0.0),
            random_state=rs,
        )))
        for i in range(samples)
    ])
    return samples

In [ ]:
inc = include_entitlements

fig, axes = plt.subplots(
    len(inc),
    len(cols),
    sharex=True,
    sharey=True,
    figsize=(12, 1*len(inc)),
)

colors = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

for i, ent in enumerate(inc):
    print(f"Fitting model for {ent}")
    to_fit = joined.dropna(subset=cols+[ent])
    samples = fit_model(
        to_fit[cols],
        to_fit[ent],
        norm=1,
        samples=100
    )
    for j, var in enumerate(cols):
        ax = axes[i, j]

        # Add a zero line
        ax.axvline(0, linewidth=2, alpha=0.3, color="k", linestyle="dotted")
        #ax.set_ylim(-0.5, 0.5)
        
        # Remove a lot of unnecessary labels and ticks on the subplots
        ax.tick_params(which="both", right=False, left=False)
        ax.set_yticklabels([])     
        for spine in ax.spines:
            ax.spines[spine].set_visible(False)

        if j == 0:
            ax.set_ylabel(ent)
        if i == 0:
            ax.set_xlabel(var)
            ax.xaxis.set_label_position("top")
        elif i == len(inc)-1:
            ax.set_xlabel(var)
            ax.xaxis.set_label_position("bottom")
            ax.tick_params(which="both", bottom=False)
        else:
            ax.tick_params(which="both", bottom=False)

        parts = ax.violinplot(samples[:, j], vert=False, showextrema=False)
        for pc in parts['bodies']:
            pc.set_color(colors[j])

fig.show()

In [ ]:
fig.savefig("entitlements.png", bbox_inches="tight", dpi=500)

In [ ]:
def plot_model(variables, target, norm=1):
    t = (target/norm).replace([numpy.nan, numpy.inf], 0.0)
    model.fit(variables, t)
    fig, axes = plt.subplots(1, len(variables.columns), figsize=(12, 3))
    for i, var in enumerate(variables.columns):
        ax = axes[i]
        ax.scatter(variables[var], target)
        #ax.scatter(variables[var], model.predict(variables))
        ax.set_xlabel(var)
        if i == 0:
            ax.set_ylabel("Count")
    plt.close(fig) # prevent double plotting
    return fig

In [ ]:
def plot_map(variables, target, norm=1):
    t = (target/norm).replace([numpy.nan, numpy.inf], 0.0)
    model.fit(variables, t)
    gdf = geopandas.GeoDataFrame(
        variables.assign(
            predict=model.predict(variables),
            target=t,
            geometry=joined.geometry,
        )
    )
    vmax = gdf.target.max()
    vmin = gdf.target.min()
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    gdf.plot(column="target", cmap="plasma", ax=ax[0])
    gdf.plot(column="predict", cmap="plasma", ax=ax[1])
    ax[0].axis("off")
    ax[1].axis("off")
    ax[0].set_title(f"Number of {target.name}")
    ax[1].set_title(f"Predicted number of {target.name}")
    plt.close(fig)
    return fig

In [ ]:
output = ipywidgets.Output()
dropdown = ipywidgets.Dropdown(options=inc)
norm = joined.pop_total
#norm = 1
def on_selection(*args):
    with output:
        output.clear_output(wait=True)
        to_plot = joined.dropna(subset=cols+[dropdown.value])
        display(
            plot_map(to_fit[cols], to_fit[dropdown.value], norm),
            plot_model(to_fit[cols], to_fit[dropdown.value], norm)
        )
dropdown.observe(on_selection, names=["value"])

on_selection()
display(dropdown, output)